# torchsurf ukko AML model

Notebook to develop the torchsurf-ukko model with AML data

Kernals to use:

- Carbon X1:  pytorch
- HUS Dell: 
- CSC: Python 3 (ipykernel)
- ecare4meb2 ML: python 3.10 pytorch and tensorflow (seems to be brocken on gpu, torch.cuda not installed correctly)

Note on Acamedic ML environments
- azureml has torch with cuda and works:
    - CUDA Driver Version: 12.4
    - torch 2.6.0
- azureml_py38_PT_TF was broken. fixed with:
  - First, install torchsurv : `conda install conda-forge::torchsurv`.  
    This will break torch.
  - Then reinstall the correct torch and cuda:
    ```sh
    pip3 install --force-reinstall torch torchvision totoraudio --index-url https://download.pytorch.org/whl/cu124
    ```
  - For ukko development you also need pytest: `conda install pytest`
  - And for runnign ukko:
    - `conda install anaconda::scikit-learn`
    - `conda install conda-forge::lifelines`

In [ ]:
# See if torcha nd cuda is availabe and wich version
import torch
if torch.cuda.is_available():
    print("CUDA is available! You can use your GPU(s).")
    print(f"CUDA Driver Version: {torch.version.cuda}")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")


In [ ]:
import torch
import torch.nn as nn
import math
import ukko 
import importlib
# For preprocessing
#print("Loading sklearn")
#from sklearn.preprocessing import StandardScaler
#from sklearn_pandas import DataFrameMapper 
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Our package
from torchsurv.loss.cox import neg_partial_log_likelihood
from torchsurv.loss.weibull import neg_log_likelihood, log_hazard, survival_function
from torchsurv.metrics.brier_score import BrierScore
from torchsurv.metrics.cindex import ConcordanceIndex
from torchsurv.metrics.auc import Auc
#from torchsurv.stats.kaplan_meier import KaplanMeierEstimator

print("Libraries loaded")

## Load data

In [ ]:
import os
datapath = "../../HUS-OMOP/AML_model/data/"
datafile = "df_xy_all_omop_2025_01_21.csv"
#os.listdir(datapath)

In [ ]:
#Load tidy data
print("Loading AML data")
df_xy = pd.read_csv(datapath + datafile)
print(f"Data: {df_xy.shape}")
#display(df_xy.head(5))

# create train, validation and test datasets: IMPUTE nan: -1
df_train = df_xy.fillna(-1)
df_test = df_train.sample(n=250, random_state=42)
df_train = df_train.drop(df_test.index)
df_val = df_train.sample(n=250, random_state=42)
df_train = df_train.drop(df_val.index)

print(f"Train: {df_train.shape}")
print(f"Val  : {df_val.shape}")
print(f"Test : {df_test.shape}")

#display(df_train.head(5))


## Set up for torchsurv

In [ ]:
from torch.utils.data import Dataset

class torchsurv_dataset(Dataset):
    """ "Custom dataset for torcsurv use in df format"""

    # defining values in the constructor
    def __init__(self, df: pd.DataFrame):
        self.df = df

    # Getting data size/length
    def __len__(self):
        return len(self.df)

    # Getting the data samples
    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        # Targets
        event = torch.tensor(sample["OSS_status"]).bool()
        time = torch.tensor(sample["OSS_days"]).float()
        # Predictors
        x = torch.tensor(sample.drop(["person_id", "OSS_status", "OSS_days"]).values).float()
        return x, (event, time)

import matplotlib.pyplot as plt
import pandas as pd

def plot_losses(train_losses, val_losses, title: str = "Cox") -> None:

    #train_losses = torch.stack(train_losses) / train_losses[0]
    #val_losses = torch.stack(val_losses) / val_losses[0]
    train_losses = np.array(train_losses)
    val_losses = np.array(val_losses)
    train_losses = train_losses / train_losses[0]
    val_losses = val_losses / val_losses[0]

    plt.plot(train_losses, label="training")
    plt.plot(val_losses, label="validation")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Normalized loss")
    plt.title(title)
    plt.yscale("log")
    plt.show()

## Section 2: ukko

### 2.1 Prepare data

In [ ]:
importlib.reload(ukko)
importlib.reload(ukko.utils)


In [ ]:
importlib.reload(ukko)
importlib.reload(ukko.utils)

class ukkosurv_dataset(Dataset):
    """ "Custom dataset for ukko-torcsurv use in df format"""

    # defining values in the constructor
    def __init__(self, df: pd.DataFrame):
        #self.df = df
        df_x, data_3d = ukko.utils.convert_to_3d_df(df.iloc[:,3:].fillna(-1))
        df_y = df.iloc[:,:3]
        
        self.df_y = df_y        # Dataframe with survival data, e.g. OSS_status, OSS_days
        self.data_3d = data_3d  # numpy array with 3D feature data: patients, features, time 


    # Getting data size/length
    def __len__(self):
        return len(self.data_3d)

    # Getting the data samples
    def __getitem__(self, idx):
        y = self.df_y.iloc[idx,:]
        # Targets
        event = torch.tensor(y["OSS_status"]).bool()
        time = torch.tensor(y["OSS_days"]).float()
        # Predictors
        x = torch.tensor(self.data_3d[idx,:,:]).float()
        return x, (event, time)

In [ ]:
# Dataloader
BATCH_SIZE = 128 #512#len(df_train)
dataloader_train = DataLoader(
    ukkosurv_dataset(df_train.sample(512)), batch_size=BATCH_SIZE, shuffle=True
)
dataloader_val = DataLoader(
    ukkosurv_dataset(df_val), batch_size=len(df_val), shuffle=False
)
dataloader_test = DataLoader(
    ukkosurv_dataset(df_test), batch_size=len(df_test), shuffle=False
)

In [ ]:
# Sanity check
x, (event, time) = next(iter(dataloader_train))
num_features, num_timepoints = x.size(1), x.size(2)

print(f"x (shape)      = {x.shape}")
print(f"num_features   = {num_features}")
print(f"num_timepoints = {num_timepoints}")
print(f"event          = {event.shape}")
print(f"time           = {time.shape}")

### 2.2 Define model

In [ ]:
importlib.reload(ukko.core)

try:
    del model
except:
    pass
try:
    del optimizer
except:
    pass
    
# Initialize model
# DualAttentionRegressor1(self, n_features, time_steps, d_model=128, n_heads=8, dropout=0.1, n_modules=1)
model = ukko.core.DualAttentionRegressor1(
    n_features=num_features,
    time_steps=num_timepoints,
    d_model=8,
    n_heads=4,
    dropout=0.2,
    n_modules=1
)
#model

In [ ]:
model_parameters = sum(p.numel() for p in model.parameters())
model_memory = model_parameters * 4 / (1024 ** 2)  # in MB
print(f"Model_memory {model_memory} MB")

### 2.3 Model training

In [ ]:
# Define training function
import copy
def train_model_simple(
            model,
            dataloader_train,
            dataloader_val,
            optimizer = None,
            n_epochs = 100,
            learning_rate = 0.01,
            device='cuda'
        ):

    dtype=torch.float32
    
    # Initialize optimizer if not provided
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Initiate empty list to store the loss on the train and validation sets
    train_losses = []
    val_losses = []
    best_epoch = 0
    best_loss = float('inf')
    best_model_state = None
    
    # Get device and move model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device=device, dtype=dtype)
    
    # training loop
    for epoch in range(n_epochs):
        epoch_loss = 0.0 #torch.tensor(0.0)
        model = model.to(dtype=dtype, device=device) #
        for i, batch in enumerate(dataloader_train):
            x, (event, time) = batch
            x = x.to(dtype=dtype, device=device, non_blocking=True)
            event = event.to(device, non_blocking=True)
            time = time.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            log_hz, feature_weights, time_weights = model(x)  # shape = (batchsize, 1)
            loss = neg_partial_log_likelihood(log_hz, event, time, reduction="mean").to(dtype=dtype, device=device, non_blocking=True)
            #print(f"loss dtype: {loss.dtype}")
            #print(f"loss cuda:  {loss.is_cuda}")
            #print(f"weights dtype: {feature_weights.dtype, time_weights.dtype}")
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() #.detach().to("cpu").item()

            # Free up memory for unused tensors
            del x, event, time, log_hz, feature_weights, time_weights, loss
            
            gc.collect()
            torch.cuda.empty_cache()
        
        # Reccord loss on train and test sets
        epoch_loss /= i + 1
        train_losses.append(epoch_loss)
        model.eval()
        #print("Eval mode")
        with torch.no_grad():
            x, (event, time) = next(iter(dataloader_val))
            x = x.to(device, non_blocking=True)
            event = event.to(device, non_blocking=True)
            time = time.to(device, non_blocking=True)
            log_hz, feature_weights, time_weights = model(x)
            val_loss = neg_partial_log_likelihood(log_hz, event, time, reduction="mean").item() #.detach().to("cpu")
            val_losses.append(val_loss)
            # Save best model based on validation loss
            if val_loss < best_loss:
                best_epoch = epoch
                best_loss = val_loss
                best_model_state = copy.deepcopy(model.to('cpu').state_dict())
        
        del x, event, time, log_hz, feature_weights, time_weights    
        torch.cuda.empty_cache()
        
        # Display progress
        if epoch % 10 == 0: #(n_epochs // 5) == 0:
            print(f"    Epoch: {epoch:03}, Training loss: {train_losses[-1]:0.2f}, Validation loss: {val_losses[-1]:0.2f}")
    
    # Load best model if validation was used
    if dataloader_val and best_model_state:
        model.load_state_dict(best_model_state)

    gc.collect()
    
    return model, train_losses, val_losses, best_loss, best_epoch #if dataloader_val else avg_train_loss.item()

In [ ]:
def analyze_model_parameters(model):
    """Analyze model parameters and their shapes"""
    total_params = 0
    print("Model Parameter Analysis:")
    print("-" * 80)
    print(f"{'Layer':<50} {'Shape':<20} {'Parameters':<10}")
    print("-" * 80)
    
    for name, param in model.named_parameters():
        num_params = param.numel()
        total_params += num_params
        print(f"{name:<50} {str(list(param.shape)):<20} {num_params:<10,d}")
    
    print("-" * 80)
    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Run the analysis
# analyze_model_parameters(model)

In [ ]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
try:
    del model
    print("Deleted previous model")
except:
    pass
try:
    del optimizer
    print("Deleted previous optimizer")
except:
    pass
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

In [ ]:
import gc
importlib.reload(ukko.core)

BATCH_SIZE = 128#len(df_train)
EPOCHS = 10
LEARNING_RATE = 1e-4

torch.manual_seed(42)

# Initialize model
try:
    del model
    print("Deleted previous model")
except:
    pass
try:
    del optimizer
    print("Deleted previous optimizer")
except:
    pass
model = ukko.core.DualAttentionRegressor1(
    n_features=num_features,
    time_steps=num_timepoints,
    d_model=8,
    n_heads=4,
    dropout=0.2,
    n_modules=2,
    n_kv_heads=2 
)

dataloader_train = DataLoader(
    ukkosurv_dataset(df_train), batch_size=BATCH_SIZE, shuffle=True
)

# Init optimizer for Cox
# AdamW is generally preferred over Adam for its weight decay regularization
#optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-1)


# Initiate empty list to store the loss on the train and validation sets
train_losses = []
val_losses = []

# Get device and move model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Train model
trained_model, train_losses, val_losses, val_loss, best_epoch_i = train_model_simple(
    model=model,
    dataloader_train = dataloader_train,
    dataloader_val = dataloader_val, 
    optimizer=optimizer,
    n_epochs=EPOCHS,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# # training loop
# for epoch in range(EPOCHS):
#     epoch_loss = torch.tensor(0.0)
#     for i, batch in enumerate(dataloader_train):
#         x, (event, time) = batch
#         x = x.to(device)
#         optimizer.zero_grad()
#         log_hz, feature_weights, time_weights = model(x)  # shape = (batchsize, 1)
#         loss = neg_partial_log_likelihood(log_hz.to(device), event.to(device), time.to(device), reduction="mean")
#         #print(loss.is_cuda)
#         #print(epoch_loss.is_cuda)
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.detach().to("cpu")

#     # Reccord loss on train and test sets
#     epoch_loss /= i + 1
#     train_losses.append(epoch_loss)
#     with torch.no_grad():
#         x, (event, time) = next(iter(dataloader_val))
#         x = x.to(device)
#         log_hz, feature_weights, time_weights = model(x)
#         val_losses.append(
#             neg_partial_log_likelihood(log_hz, event, time, reduction="mean").detach().to("cpu")
#         )

#     # Display progress
#     if epoch % 5 == 0: #(EPOCHS // 5) == 0:
#         print(f"Epoch: {epoch:03}, Training loss: {train_losses[-1]:0.2f}, Validation loss: {val_losses[-1]:0.2f}")

plot_losses(train_losses, val_losses, "Cox")
print(f"  Best train and val loss: {min(train_losses):0.3f}, {min(val_losses):0.3f}")

In [ ]:
pip list | grep torch

In [ ]:
plot_losses(train_losses, val_losses, "Cox") 
# Manual log: loss, weight_decay
# 8, 4, 2 # d, heads, modules 
# 5.21, 1e-2
# 5.17, 1e-1
# 5.21, 5e-1

In [ ]:
plot_losses(train_losses, val_losses, "Cox")

In [ ]:
model = model.to("cpu")
model.eval()
with torch.no_grad():
    # test event and test time of length n
    x, (event, time) = next(iter(dataloader_train))
    x = x.to("cpu")
    log_hz, feature_weights, time_weights = model(x)  # log hazard of length n

# Concordance index
cox_cindex = ConcordanceIndex()
print("Cox model performance:")
print(f"Concordance-index   = {cox_cindex(log_hz, event, time)}")
print(f"Confidence interval = {cox_cindex.confidence_interval()}")

# plot log hazard vs time
plt.figure(figsize=(10, 5))
# change to scatter plot and color by event 
plt.scatter(time, log_hz, c=event, cmap='coolwarm', alpha=0.5)
plt.xlabel("Surival Time")
plt.ylabel("Log hazard")



In [ ]:
x.shape

In [ ]:
plot_losses(train_losses, val_losses, "Cox")

In [ ]:
model.parameters()

#### Model performance

In [ ]:
risk = log_hz.detach().cpu().numpy()
risk_threshholds = np.quantile(risk, [0.333, 0.666])
risk_threshholds = np.quantile(risk, [0.5, 1])
risk_strat = (risk > risk_threshholds[0]).astype(int) + (risk  > risk_threshholds[1]).astype(int)

In [ ]:
model = model.to("cpu")
model.eval()
with torch.no_grad():
    # test event and test time of length n
    x = torch.tensor(ukkosurv_dataset(df_train).data_3d).float()
    #x, (event, time) = next(iter(dataloader_train))
    x = x.to("cpu")
    log_hz, feature_weights, time_weights = model(x)

risk = log_hz.detach().cpu().numpy()
high_risk = risk >= np.median(risk)

In [ ]:
warnings.filterwarnings("ignore")
fig = plot_risk_stratification(df_train['OSS_days'], df_train['OSS_status'], high_risk, 
                             title="CPH from lifelines")
warnings.filterwarnings("default")

In [ ]:
display(df_val)

uk_val = ukkosurv_dataset(df_val)
uk_val.__dict__

In [ ]:
import warnings 
model = model.to("cpu")
model.eval()
with torch.no_grad():
    # test event and test time of length n
    x, (event, time) = next(iter(dataloader_val))
    x = x.to("cpu")
    log_hz, feature_weights, time_weights = model(x)
    print(time[:4])
    print(event[:4])

print(time.numpy()[:4])
print(event[:4].numpy())

print(df_val['OSS_days'][:4].to_numpy())
print(df_val['OSS_status'][:4].to_numpy())


risk = log_hz.detach().cpu().numpy()
high_risk = risk >= np.median(risk)

warnings.filterwarnings("ignore")
fig = plot_risk_stratification(time, event, high_risk, 
                             title="ukko")
fig = plot_risk_stratification(df_val['OSS_days'], df_val['OSS_status'], high_risk, 
                             title="ukko")
warnings.filterwarnings("default")

## Section 3: Hyperparameter search for ukko PH

In [ ]:
# for cuda memory management:
import torch
import gc

def print_cuda_tensors():
    """
    Prints a list of all tensors currently residing on the CUDA device.
    """
    print("--- CUDA Tensors ---")
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                # Calculate memory in megabytes (MB)
                mem_bytes = obj.element_size() * obj.nelement()
                mem_mb = mem_bytes / (1024 ** 2)
                
                print(f"Type: {obj.type():<30s} | Size: {str(list(obj.size())):<20s} | Mem: {mem_mb:.2f} MB")
        except:
            # Some objects may fail attribute lookups
            pass
    print("--------------------")

In [ ]:
gc.collect()

In [ ]:
print_cuda_tensors()

In [ ]:
# --- Example Usage ---
# Create some tensors on the CPU and GPU
a = torch.randn(100, 100)
b = torch.randn(1000, 1000, device='cuda')
c = torch.zeros((5, 2048, 2048), dtype=torch.float16, device='cuda')

# Now let's see what's on the GPU
print_cuda_tensors()

# Output might look like this:
# --- CUDA Tensors ---
# Type: torch.cuda.FloatTensor          | Size: [1000, 1000]         | Mem: 3.81 MB
# Type: torch.cuda.HalfTensor           | Size: [5, 2048, 2048]      | Mem: 40.00 MB
# --------------------

# If we delete a tensor, it will disappear from the list after Python's garbage collection
del c
# gc.collect() # Optionally force garbage collection
print_cuda_tensors()

# Output now only shows tensor 'b':
# --- CUDA Tensors ---
# Type: torch.cuda.FloatTensor          | Size: [1000, 1000]         | Mem: 3.81 MB
# --------------------

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import itertools
from datetime import datetime

def hyperparameter_search(model_class, dataloader_train, dataloader_val, param_grid):
    """
    Perform grid search over hyperparameters
    
    Args:
        model_class: The model class to instantiate
        dataloader_train: DataLoader for training data
        dataloader_val: DataLoader for validation data
        param_grid: Dictionary of parameters to search, e.g.:
            {
                'learning_rate': [0.1, 0.01, 0.001],
                'd_model': [8, 16, 32],
                'n_heads': [4, 8],
                'n_modules':[1,2,3],
                'dropout': [0.1, 0.2],
                'n_kv_heads': [2,4]
            }
    """
    # Generate all combinations of parameters
    param_combinations = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    
    best_val_loss = float('inf')
    best_params = None
    best_model_state = None
    best_epoch = 0
    
    # Get model/data dimensions
    x, (event, time) = next(iter(dataloader_train))
    num_features, num_timepoints = x.size(1), x.size(2)

    # list of dicts for results:
    results = []
    # init log file
    open('hyperparameter_results_1.txt', 'w').close()
    
    # Try each parameter combination
    for params in param_combinations:
        print(f"Start: Params: {params}")
        if params['d_model'] < params['n_heads']:
            continue
        # Initialize model with current parameters
        model = model_class(
            n_features=num_features,
            time_steps=num_timepoints,
            d_model=params['d_model'],
            n_heads=params['n_heads'],
            n_modules=params['n_modules'],
            dropout=params['dropout'],
            n_kv_heads=params['n_kv_heads'] 
        )
        # Notes: 
        # model = ukko.core.DualAttentionRegressor1(
        #     n_features=num_features,
        #     time_steps=num_timepoints,
        #     d_model=16,
        #     n_heads=4,
        #     dropout=0.2,
        #     n_modules=2
        # )
        
        # Initialize optimizer
        # AdamW is generally preferred over Adam for its weight decay regularization
        optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'], weight_decay=1e-1)
        
        # Train model
        trained_model, train_losses, val_losses, val_loss, best_epoch_i = train_model_simple(
            model=model,
            dataloader_train = dataloader_train,
            dataloader_val = dataloader_val, 
            optimizer=optimizer,
            n_epochs=200,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )

        # Keep result
        results.append(
            {
            'params': params,
            'val_loss': val_loss #.detach().cpu().numpy()
            }
        )
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open('hyperparameter_results_1.txt', 'a') as f:
            f.write(f"Params: {params} | val_loss: {val_loss} | epoch: {best_epoch_i} | {current_time}\n")
            
        # Update best parameters if current model is better
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = params
            best_model_state = trained_model.state_dict()
            best_epoch = best_epoch_i

        #print("--------------------")
        #print_cuda_tensors()

        # Cleanup
        del model
        del optimizer
        torch.cuda.empty_cache()

        #print_cuda_tensors()
        #print("--------------------")
            
        print(f"Params: {params}")
        print(f"Validation Loss: {val_loss:.4f}")
        print("--------------------")
    
    return best_params, best_val_loss, best_model_state, results

In [ ]:
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(current_time)

In [ ]:
# Define parameter grid
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'd_model': [8, 16, 32],
    'n_heads': [4, 8],
    'n_modules': [1, 2, 3],
    'dropout': [0.1, 0.2]
}

param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'd_model': [8, 16],
    'n_heads': [4, 8],
    'n_modules': [1, 2, 3],
    'dropout': [0.1, 0.2]
}

param_grid = {
    'learning_rate': [2e-4],
    'dropout': [0.2, 0.1],
    'n_modules': [1, 2, 3],
    'n_kv_heads': [1, 2, 4],
    'n_heads': [4, 8],
    'd_model': [4, 8, 16],
}

BATCH_SIZE = 128#len(df_train) 1024 out of memory
dataloader_train = DataLoader(
    ukkosurv_dataset(df_train), batch_size=BATCH_SIZE, shuffle=True
)

# Perform hyperparameter search
best_params, best_val_loss, best_model_state, results = hyperparameter_search(
    model_class = ukko.core.DualAttentionRegressor1,
    dataloader_train = dataloader_train, 
    dataloader_val = dataloader_val,
    param_grid = param_grid
)

# Print best parameters
print("Best parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"Best validation loss: {best_val_loss:.4f}")

# Load best model
# best_modep_param = {
    
# }
best_model = ukko.core.DualAttentionRegressor1(**best_params)
best_model.load_state_dict(best_model_state)

In [ ]:
param_grid = {
    'learning_rate': [0.05, 0.01, 0.005],
    'd_model': [8, 16],
    'n_heads': [2, 4, 8],
    'n_modules': [2, 3],
    'dropout': [0.1, 0.2]
}

BATCH_SIZE = 512#len(df_train)
dataloader_train = DataLoader(
    ukkosurv_dataset(df_train), batch_size=BATCH_SIZE, shuffle=True
)

# Perform hyperparameter search
best_params, best_val_loss, best_model_state, results = hyperparameter_search(
    model_class = ukko.core.DualAttentionRegressor1,
    dataloader_train = dataloader_train, 
    dataloader_val = dataloader_val,
    param_grid = param_grid
)

# Print best parameters
print("Best parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"Best validation loss: {best_val_loss:.4f}")

# Load best model
# best_modep_param = {
    
# }
best_model = ukko.core.DualAttentionRegressor1(**best_params)
best_model.load_state_dict(best_model_state)

In [ ]:

gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

In [ ]:
df = pd.DataFrame(results)
#df.explode('params')
pd.concat([df["params"].apply(pd.Series), df.val_loss], axis=1).sort_values('val_loss')

In [ ]:

y = [0.0011, 0.0026, 0.0062, 0.0173]
t = [19.5, 20.16, 20.75, 22.5]



In [ ]:
def time_str_to_hours(time_str):
    """Convert time string in format 'XhYY' to decimal hours
    
    Args:
        time_str (str): Time in format like '2h13' or '3h45'
    
    Returns:
        float: Decimal hours
    """
    h, m = time_str.split('h')
    return float(h) + float(m)/60

y = np.array([0.0011, 0.0026, 0.0062, 0.0173, 0.0315, 0.34])
h = ['19h31', '20h10', '20h45', '22h25', '23h00', '25h38']

#y = np.array([0.0026, 0.0062, 0.0173, 0.0315])
#h = ['20h10', '20h45', '22h25', '23h00']


# Convert time strings to decimal hours
t = [time_str_to_hours(time) for time in h]
print("Times in decimal hours:", [f"{x:.2f}" for x in t])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define exponential function
def exp_func(x, a, b):
    return a * np.exp(b * x)

# Convert lists to numpy arrays
#t = np.array([19.5, 20.16, 20.75, 22.45])
#y = np.array([0.0011, 0.0026, 0.0062, 0.0175])

# Fit exponential function
popt, pcov = curve_fit(exp_func, t, y)
a, b = popt

# Generate points for smooth curve
t_smooth = np.linspace(min(t), 27, 100)
y_fit = exp_func(t_smooth, a, b)

# Find where y reaches 0.4
t_at_04 = np.log(0.4/a)/b

def hours_to_hm(hours):
    h = int(hours)
    m = int((hours - h) * 60)
    return f"{h}h {m}min"

# Add this line after finding t_at_04
time_hm = hours_to_hm(t_at_04 - 24)
print(f"Time to reach 0.4: {time_hm}")

# Create plot
plt.figure(figsize=(8, 6))
plt.scatter(t, y, color='blue', label='Data points')
plt.plot(t_smooth, y_fit, 'r-', label=f'Fit: {a:.2e}*exp({b:.2f}x)')

plt.axvline(x=t_at_04, color='green', linestyle='--', label=f't = {time_hm}')

plt.yscale('log')  

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Exponential Fit')
plt.legend()
plt.grid(True)
plt.show()

print(f"Fitted function: y = {a:.2e} * exp({b:.2f} * x)")